### Load data and pre-trained model

In [ ]:
import boto3

In [ ]:
from pathlib import Path
PATH = Path('/Users/gjke/Documents/uni/faktual/s3')

In [ ]:
BUKET_NAME = 'gpt2-finetuning'

In [ ]:
def download_directory_from_s3(bucket_name, remote_directory_name, local_destination ):
    s3_resource = boto3.resource('s3')
    bucket = s3_resource.Bucket(bucket_name) 
    for object in bucket.objects.filter(Prefix = remote_directory_name):
        print(object, os.path.dirname(object.key))
        path, filename = os.path.split(object.key)
        if filename == '':
            continue
        # boto3 s3 download_file will throw exception if folder not exists
        try:
            os.makedirs(os.path.join(local_destination,path)) 
        except FileExistsError:
            pass
        bucket.download_file(object.key, os.path.join(local_destination, object.key))
        

In [ ]:
download_directory_from_s3(BUKET_NAME, 'swissdata', PATH)


In [ ]:
download_directory_from_s3(BUKET_NAME, 'de345-root', PATH)

### Add <tldr> token

In [ ]:
from lm.inference import ModelWrapper
import sentencepiece_model_pb2 as model

In [ ]:
PATH_TO_SP_MODEL = os.path.join(PATH, 'de345-root' 'sp.model')


In [ ]:
m = model.ModelProto()
m.ParseFromString(open(PATH_TO_SP_MODEL, 'rb').read())

In [ ]:
first_piece = m.pieces[0]

In [ ]:
from copy import copy
new_piece = copy(first_piece)

In [ ]:
new_piece.Clear()
new_piece.piece = '<tldr>'
new_piece.score = 0.0
new_piece.type = 4

In [ ]:
m.pieces.pop()
m.pieces.append(new_piece)

In [ ]:
len(m.pieces), m.pieces[-1]

In [ ]:
with open(PATH_TO_SP_MODEL , 'wb') as f:
    f.write(m.SerializeToString())

### Prepare the data

In [ ]:
from pathlib import Path

In [ ]:
PATH_TO_DATA = os.path.join(PATH, 'swissdata')

In [ ]:
PATH_TO_MODEL = os.path.join(PATH, 'de345-root')

#### Reformat train.csv using <tldr\> token

In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv(PATH_TO_DATA / 'data_train.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
TLDR = '<tldr>'

In [ ]:
if not os.path.exists(PATH_TO_DATA / 'train'):
    os.mkdir(PATH_TO_DATA / 'train')
with open(PATH_TO_DATA / 'train/data_train_tldr.txt', 'w') as f:
    for row in train_df.loc[:int(train_df.shape[0]*0.85)].itertuples(index=False):
        f.write(row[0] + TLDR + row[1] + "\n")

In [ ]:
if not os.path.exists(PATH_TO_DATA / 'valid'):
    os.mkdir(PATH_TO_DATA / 'valid')
with open(PATH_TO_DATA / 'valid/data_val_tldr.txt', 'w') as f:
    for row in train_df.loc[int(train_df.shape[0]*0.85):int(train_df.shape[0]*0.95)].itertuples(index=False):
        f.write(row[0] + TLDR + row[1] + "\n")

In [ ]:
if not os.path.exists(PATH_TO_DATA / 'test'):
    os.mkdir(PATH_TO_DATA / 'test')
with open(PATH_TO_DATA / 'test/data_test_tldr.txt', 'w') as f:
    for row in train_df.loc[int(train_df.shape[0]*0.95):].itertuples(index=False):
        f.write(row[0] + TLDR + row[1] + "\n")

In [ ]:
del train_df

### Encode the data

In [ ]:
!sp-encode $PATH_TO_DATA $PATH_TO_MODEL/sp.model $PATH_TO_DATA/encoded

### Start training 

In [ ]:
!gpt-2 $PATH_TO_MODEL $PATH_TO_DATA/encoded $PATH_TO_MODEL/sp.model --n_embed=1024 --n_ctx=1024 --n_head=16 --n_layer=24 --n_hidden=1024 --batch_size=3 --gradient_checkpointing=0 --save_every=5000